*Bugs and how to circumvent them.* 

*Last update:* 2019-05-31

# COBALT bugs

Here are some reference notes as I wade through annual runs with COBALT.

## Missing hioekg_dia.nc and hioekg_avg.nc files

These files are not being written. I wonder if this is due to the specification in hioekg.h that leaves them undefined. To test this, I am changing the run specs and defining the files. I am setting the model to run for 3 days in Jan. 2013.

```hioekg.h```: Changed from:

```
#define BIOLOGY
#define BIO_COBALT
#if defined BIO_COBALT
# define OPTIC_MANIZZA
# define COBALT_MINERALS
# define COBALT_PHOSPHORUS
# define COBALT_IRON
# define COBALT_CARBON
# undef DIAGNOSTICS        /* diagnostics files contain Chl-a and .. */
# undef DIAGNOSTICS_BIO   /* derived biol. val. needed for restart and analysis*/
```

Changed to:

```
#define BIOLOGY
#define BIO_COBALT
#if defined BIO_COBALT
# define OPTIC_MANIZZA
# define COBALT_MINERALS
# define COBALT_PHOSPHORUS
# define COBALT_IRON
# define COBALT_CARBON
# define DIAGNOSTICS        /* diagnostics files contain Chl-a and .. */
# define DIAGNOSTICS_BIO   /* derived biol. val. needed for restart and analysis*/
```
--------

``ocean.in``: Changed from 30 days ((30 days * 86400 sec in a day) / DT=sec per step=10):

```
 Output history, quicksave, average, and diagnostic files parameters.

! Time-Stepping parameters.

      NTIMES == 259200
```
to 3 days ((3 * 86400) / 10):
```
 ! Time-Stepping parameters.

      NTIMES == 25920
```
--------

```spinup.sh```: Changed from:

```
ROMS_END_DAY=5112

ROMS_WORK_DIR=${PBS_O_WORKDIR}/work
ROMS_OUT_DIR=${PBS_O_WORKDIR}/output
ROMS_RUN_DAYS=30 
```

Changed to:

```
ROMS_END_DAY=4752

ROMS_WORK_DIR=${PBS_O_WORKDIR}/work_dia_test
ROMS_OUT_DIR=${PBS_O_WORKDIR}/output_dia_test
ROMS_RUN_DAYS=3 
```

*Update 5/30*: The above protocol was not successful. The changes above remain in place and I am also changing the following:

```ocean.in```: Changed from:
```
LDEFOUT == T
        NHIS == 8640
     NDEFHIS == 0
        NQCK == 0
     NDEFQCK == 0
      NTSAVG == 1
        NAVG == 8640
     NDEFAVG == 0
      NTSDIA == 1
        NDIA == 8640
     NDEFDIA == 0
```

Changed to:

```
LDEFOUT == T
        NHIS == 8640
     NDEFHIS == 0
        NQCK == 0
     NDEFQCK == 0
      NTSAVG == 1
        NAVG == 8640
     NDEFAVG == 8640
      NTSDIA == 1
        NDIA == 8640
     NDEFDIA == 8640
```
*Update 5/31*: The above protocol was also not successful. I am undoing the 5/30 part 2 changes above and reverting ocean.in to:

```
LDEFOUT == T
        NHIS == 8640
     NDEFHIS == 0
        NQCK == 0
     NDEFQCK == 0
      NTSAVG == 1
        NAVG == 8640
     NDEFAVG == 0
      NTSDIA == 1
        NDIA == 8640
     NDEFDIA == 0
```

Keeping all else the same aside from using cycle_roms_plus* (copied from /HIIG_Cobalt_2010).

## Creating .nc files with ``setup_inner_*`` scripts

``_frc``: General input error when...

```python
start_day = 4749 # 1-1-2013
end_day = 5113 # 12-31-2013
```
**Solution**: The script works fine, but hits a wall when a timeseries > 250 days is called from the PacIOOS server. Call dates only in segments shorter than this approximate limit, then concatenate the renamed resulting mini-files:
```
ncrcat -h frc-wrfo-1.nc frc-wrfo-2.nc frc-wrfo.nc
ncrcat -h frc-cobalt-1.nc frc-cobalt-2.nc frc-cobalt.nc
ncrcat -h river-hioekg-1.nc river-hioekg-2.nc river-hioekg.nc
```

## Duplicate dates at end/start of months

Only the bry-phy portion of setup_inner_nest.py produces problematic files (i.e., a duplicate of 00:00 at the start of each month). 

Note that cycle_roms reproduces this error during the monthly COBALT runs, so you will need to check for and eliminate duplicates 1) during the generation of the input netCDF files (...-bry-phy.nc) ***and*** during model output post-processing.

1) First, I set setup_inner_nest.py to run from 
```
rst = 4743
start day = 4749
end day = 4779
```
I did not edit this file.

2) Next, I set setup_inner_nest.py to run from 

```
rst, start = 4779
end day = 4809
```
3) The values are zero indexed, so I eliminated the first value, the duplicate of 00:00 on Jan 31, and kept everything else. 

```
ncks -O -d bry_time,1,241 hioekg-bry-phy-4779-4809.nc hioekg-bry-phy-4779-4809.nc
```

4) Finally, concatenate the two files:
```
ncrcat -h hioekg-bry-phy-4749-4779.nc hioekg-bry-phy-4779-4809.nc hioekg-bry-phy-4749-4809.nc 
```
Note that spinup.sh and ocean.in direct cycle_roms to run for 30 days; even though this file contains 2 month of climatological data, cycle_roms will ingest 30 days at a time.

## Improper his-ini start time

1. For any start date, his-ini will begin at 00:00. The hioekg-bry-bio.nc file wil begin at 12:00 on the same start date. This will throw an error in the dry run.

**Solution**: Change time by one day (= 86400 seconds) and rename the file.
```bash
ncap2 -s 'ocean_time=ocean_time+86400' his-ini-04749.nc his-ini-04750.nc
```
In spinup.sh, ROMS_START_DAY = 4750 to correspond with this modification.